In [3]:
import matplotlib.pyplot as plt
import scipy.constants as cons
import pandas as pd
import numpy as np
import scipy
import cmath
import time
import os

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from qutip import *

# Column Heads for data output
columns=['Wigint', 'WLN', 'SqzFactor', 'DisFactor', 'Xrange', 'Xstep', 'Yrange', 'Ystep', 'N']

In [112]:
def myplotwigner(psi, xrange = [0,3], yrange = [-20,20], step = 0.1, title = 'Wigner Function'):
    """
    Function for plotting the Wiger function of a state which gives more control
    over the appearance of the graph than the built in qutip plot_wigner function
    """
    # Generate X and Y values from inputs
    xvec = np.arange(xrange[0], xrange[1], step)
    yvec = np.arange(yrange[0], yrange[1], step)
    X,Y = np.meshgrid(xvec, yvec)
    
    # Calculate Wigner function at specified coordinates
    W = wigner(psi, xvec, yvec)
    
    # Create Figure and Axes for the plot
    fig = plt.figure(figsize=(8,6))
    ax = Axes3D(fig) 
    
    # plot surface
    ax.plot_surface(X, Y, W, rcount= 50, ccount= 50, cmap=cm.jet, alpha=.8)
    
    # Overlay contours onto plot
    ax.contour(X, Y, W, 15, zdir='x', offset=xrange[1])
    ax.contour(X, Y, W, 15, zdir='y', offset=yrange[1])
    ax.contour(X, Y, W, 15,zdir='z', offset=0)
    
    # Label Axes appropriately
    ax.set_xlabel(r'$\rm{Re}(\alpha)$')
    ax.set_ylabel(r'$\rm{Im}(\alpha)$')
    ax.set_zlabel(r'$W(\alpha)$')
    
    # Add title
    plt.title(title)
    return fig, ax


def simps2d(Xvec, Yvec, Z):
    """
    Function to return value of the 2d simpson integration over a sample of values
    :Xvec: first axis of values at which function is evaluated
    :Yvec: second axis of values at which function is evaluated
    :Z: Function values, first indice being X and second Y
    """
    return scipy.integrate.simps(scipy.integrate.simps(Z, Xvec), Yvec)


def catstate(alpha, phi, theta, N):
    coh1 = np.cos(phi) * coherent(N, alpha)
    coh2 = np.sin(phi) * cmath.rect(1,theta) * coherent(N, -alpha)
    K = 1 + np.sin(2 * phi) * np.cos(theta) * np.exp(-2 * alpha * np.conj(alpha))
    norm = 1/np.sqrt(K)
    return norm * (coh1 + coh2)


def cubicstate(gamma, squeezing, N):
    x = position(N)
    
    expon = (1j * gamma * (x ** 3)).expm()
    sque = squeeze(N,squeezing)

    cubic = (expon * sque * basis(N,0).unit()).unit()
    return cubic


In [132]:
# Name datafile for output and if not existing create and add column headings
datafile = 'mastersimp.csv'
if not os.path.isfile(datafile):
    dfinit = pd.DataFrame(columns=columns)
    with open(datafile, 'a') as f:
        dfinit.to_csv(f)

In [ ]:
# Initializing / clearing Lists to store data to be input to pandas dataframe and exported to csv, 
# d prefix to symbolize they are used for data storage
written = False
dwig = []
dWLN = []
dsqzf = []
ddisf = []
dnumlev = []
dxrange = []
dxstep = []
dyrange =  []
dystep = []

In [64]:
# Generate states needed to perform calculations later and also initialize variables (states need to be regenerated
# if the variables preceding them are altered)

N=250 # Number of Fock 'levels' to use in approximation

vac = basis(N, 0) # Vacuum
coh = coherent(N, 1) # Coherent state
fockone = basis(N,1) # First Fock state

sqf=1 # Squeezing Factor
dif=1 # Displacement Factor

# Normalization factor
Nadd = 1 + np.sinh(sqf) ** 2 + np.abs(dif) ** 2
Nsub = np.sinh(sqf) ** 2 + np.abs(dif) ** 2

psiadd = (1/np.sqrt(Nadd)) * create(N) * displace(N,dif) * squeeze(N, sqf) * basis(N, 0) 
rhoadd = ket2dm(psiadd)

psisub = (1/np.sqrt(Nsub)) *  destroy(N) * displace(N,dif) * squeeze(N, sqf) * basis(N, 0) 
rhosub = ket2dm(psisub)

psi =  displace(N,1) * squeeze(N, 0.5) * basis(N, 0) 

In [ ]:
# Generate and Store a plot for the wigner function given the following ranges to determine efficient limits for 
# simpson integration
xrange = [0.75, 2]
yrange = [-25, 25]
fig, ax = myplotwigner(psi, xrange, yrange)

In [ ]:
# Show plot again, nb. not sure how to make this interactive
%matplotlib inline
# Set View Angle
ax.view_init(30, -110)
display(fig)

In [114]:
xstep = 1/100
ystep = 5/100

starttime = time.time()

xvec = np.arange(xrange[0], xrange[1], xstep)
yvec = np.arange(yrange[0], yrange[1], ystep)
W = wigner(psi, xvec, yvec)

endtime = time.time()

# Time taken to calculate the wigner function for the set of x and y coordinates
time = endtime - starttime

In [34]:
# Calculates the integral over the Wigner function for the ranges specified above, serves as a check to see if 
# we are approximating the integration accurately enough as should be normalised to 1.
wigtemp = simps2d(xvec, yvec, W)

# Calculates the integral of the absolute value of the Wigner funtion and then takes the log (base 2) to determine
# the Wigner Logarithmic Negativity (in base 2) for the function. This serves as a monotone/quantifier for the
# quantum non-Gaussianity of the state.
WLNtemp = np.log2(simps2d(xvec, yvec, abs(W)))
print("Integral over the Wigner function = {}".format(wigtemp), "WLN = {}".format(WLNtemp))

"""
# Uncomment and then comment block below if want to look over results before appending
if written is True:
        print('Already written to file this session, try reseting data lists first to prevent writing same data  twice')
elif written is False:
    while True:
        query = input('accept results? If first value unreasonably less than 1 reject.')
         Fl = query.lower()
         if query == '' or not Fl in ['y','n','yes','no']:
            print('Please answer with yes or no!')
         else:
            break
            
    if Fl in ['yes', 'y']:  
        # Appending Values to Corresponding lists
        dsqzf.append(sqf)
        ddisf.append(dif)
        dnumlev.append(N)
        dxrange.append(xrange)
        dxstep.append(xstep)
        dyrange.append(yrange)
        dystep.append(ystep)
        dwig.append(wigtemp)
        dWLN.append(WLNtemp)
        dtime.append(time)

        print('Data appended')

    elif Fl in ['no', 'n']:
        print('Results not appended')
"""

if written is True:
    print('Already written to file this session, try reseting data lists first to prevent writing same data  twice')

elif written is False:
    # Appending Values to Corresponding lists
    dsqzf.append(sqf)
    ddisf.append(dif)
    dnumlev.append(N)
    dxrange.append(xrange)
    dxstep.append(xstep)
    dyrange.append(yrange)
    dystep.append(ystep)
    dwig.append(wigtemp)
    dWLN.append(WLNtemp)
    dtime.append(time)
    
    print('Data appended')

0.9979677914308335


In [ ]:
data = list(zip(dwig, dWLN, dsqzf, ddisf, dxrange, dxstep, dyrange, dystep ,dnumlev, dtime))

df = pd.DataFrame(data, columns=columns)

In [1]:
df

NameError: name 'df' is not defined

In [ ]:
if written is True:
    print('Already written to file this session, try reseting data lists first to prevent writing same data  twice')
elif written is False:
    with open(datafile, 'a') as f:
        df.to_csv(f, header=False)
        written = True